Part 1


In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()

        # Define various layers here
        self.fc1 = nn.Linear(28*28, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 10)
        self.m1_fc1 = nn.Linear(28*28, 20)
        self.m1_fc2 = nn.Linear(20, 15)
        self.m1_fc3 = nn.Linear(15, 10)
        self.m3_fc1 = nn.Linear(28*28, 400)
        self.m3_fc2 = nn.Linear(400, 350)
        self.m3_fc3 = nn.Linear(350, 250)
        self.m3_fc4 = nn.Linear(250, 200)
        self.m3_fc5 = nn.Linear(200, 10)

        # This will select the forward pass function based on mode for the ConvNet.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        else:
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)

    # Baseline sample model
    def model_0(self, X):
        # ======================================================================
        # Three fully connected layers with activation

        X = torch.flatten(X, start_dim=1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)
        X = torch.sigmoid(X)

        return X

    # Baseline model. task 1
    def model_1(self, X):
        # ======================================================================
        # Three fully connected layers without activation

        X = torch.flatten(X, start_dim=1)
        # NotImplementedError
        X = self.m1_fc1(X)
        X = self.m1_fc2(X)
        X = self.m1_fc3(X)
        return X


    # task 2
    def model_2(self, X):
        # ======================================================================
        # Train with activation (use model 1 from task 1)

        X = torch.flatten(X, start_dim=1)
        X = self.m1_fc1(X)
        X = F.relu(X)
        X = self.m1_fc2(X)
        X = F.relu(X)
        X = self.m1_fc3(X)
        X = F.relu(X)
        # NotImplementedError

        return X


    # task 3
    def model_3(self, X):
        # ======================================================================
        # Change number of fully connected layers and number of neurons from model 2 in task 2
        self.fc1 = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 10)

        X = torch.flatten(X, start_dim=1)
        X = self.m3_fc1(X)
        X = F.relu(X)
        X = self.m3_fc2(X)
        X = F.relu(X)
        X = self.m3_fc3(X)
        X = F.relu(X)
        X = self.m3_fc4(X)
        X = F.relu(X)
        X = self.m3_fc5(X)
        X = F.relu(X)
        # NotImplementedError

        return X







In [2]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

X = torch.randn(16, 3, 128, 96)
print(X.shape)

torch.Size([16, 3, 128, 96])


In [ ]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
#from ConvNet import ConvNet
import argparse
import numpy as np


def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''

    # Set model to train mode before each epoch
    model.train()

    # Empty list to store losses
    losses = []
    correct = 0

    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample

        # Push data/label to correct device
        data, target = data.to(device), target.to(device)

        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()

        # Do forward pass for current set of data
        output = model(data)

        # ======================================================================
        # Compute loss based on criterion
        # ----------------- YOUR CODE HERE ----------------------
        #
        # Remove NotImplementedError and assign correct loss function.
        #loss = nn.MSELoss(model)
        #criterion = nn.MSELoss()
        loss = criterion(output, target)


        # Computes gradient based on final loss
        loss.backward()

        # Store loss
        losses.append(loss.item())

        # Optimize model parameters based on learning rate and gradient
        optimizer.step()

        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)

        # ======================================================================
        # Count correct predictions overall
        correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc



def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''

    # Set model to eval mode to notify all layers.
    model.eval()

    losses = []
    correct = 0

    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)


            # Predict for data by doing forward pass
            output = model(data)

            # ======================================================================
            # Compute loss based on same criterion as training
            loss = F.cross_entropy(output, target, reduction='mean')

            # Append loss to overall test loss
            losses.append(loss.item())

            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)

            # ======================================================================
            # Count correct predictions overall
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))

    return test_loss, accuracy


def run_main(FLAGS):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()

    # Set proper device based on cuda availability
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Torch device selected: ", device)

    # Initialize the model and send to device
    model = ConvNet(FLAGS.mode).to(device)

    # Initialize the criterion for loss computation
    # ======================================================================
    # Remove NotImplementedError and assign correct loss function.
    criterion = nn.CrossEntropyLoss()


    # Initialize optimizer type
    optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)

    # Create transformations to apply to each data sample
    # Can specify variations such as image flip, color flip, random crop, ...
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

    # Load datasets for training and testing
    # Inbuilt datasets available in torchvision (check documentation online)
    dataset1 = datasets.MNIST('./data/', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('./data/', train=False,
                       transform=transform)
    train_loader = DataLoader(dataset1, batch_size = FLAGS.batch_size,
                                shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size = FLAGS.batch_size,
                                shuffle=False, num_workers=4)

    best_accuracy = 0.0

    # Run training for n_epochs specified in config
    for epoch in range(1, FLAGS.num_epochs + 1):
        print("\nEpoch: ", epoch)
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, FLAGS.batch_size)
        test_loss, test_accuracy = test(model, device, test_loader)

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy

    print("accuracy is {:2.2f}".format(best_accuracy))

    print("Training and evaluation finished")


if __name__ == '__main__':
    # Set parameters for Sparse Autoencoder
    parser = argparse.ArgumentParser('CNN Exercise.')
    parser.add_argument('--mode',
                        type=int, default=2,
                        help='Select mode between 1-3.')
    parser.add_argument('--learning_rate',
                        type=float, default=0.01,
                        help='Initial learning rate.')
    parser.add_argument('--num_epochs',
                        type=int,
                        default=20,
                        help='Number of epochs to run trainer.')
    parser.add_argument('--batch_size',
                        type=int, default=10,
                        help='Batch size. Must divide evenly into the dataset sizes.')
    parser.add_argument('--log_dir',
                        type=str,
                        default='logs',
                        help='Directory to put logging.')

    FLAGS = None
    FLAGS, unparsed = parser.parse_known_args()

    print("Mode: ", FLAGS.mode)
    print("LR: ", FLAGS.learning_rate)
    print("Batch size: ", FLAGS.batch_size)

    run_main(FLAGS)



Mode:  2
LR:  0.01
Batch size:  10
Torch device selected:  cpu

Epoch:  1
Train set: Average loss: 1.3481, Accuracy: 28043/60000 (47%)
Test set: Average loss: 1.2446, Accuracy: 4882/10000 (49%)

Epoch:  2
Train set: Average loss: 1.2329, Accuracy: 29534/60000 (49%)
Test set: Average loss: 1.2137, Accuracy: 4954/10000 (50%)

Epoch:  3
Train set: Average loss: 1.2098, Accuracy: 29742/60000 (50%)
Test set: Average loss: 1.2073, Accuracy: 4961/10000 (50%)

Epoch:  4
Train set: Average loss: 1.1963, Accuracy: 29888/60000 (50%)
Test set: Average loss: 1.1939, Accuracy: 5004/10000 (50%)

Epoch:  5
Train set: Average loss: 1.1878, Accuracy: 29994/60000 (50%)
Test set: Average loss: 1.1897, Accuracy: 4989/10000 (50%)

Epoch:  6
Train set: Average loss: 1.1818, Accuracy: 30058/60000 (50%)
Test set: Average loss: 1.1893, Accuracy: 4982/10000 (50%)

Epoch:  7
Train set: Average loss: 1.1772, Accuracy: 30073/60000 (50%)
Test set: Average loss: 1.1903, Accuracy: 4987/10000 (50%)

Epoch:  8
Train set

Task 1: Model 1 (learning rate = 0.01, epochs=20, batch_size=10)
Train loss: 0.2657 | Accuracy: 92%
Test loss: 0.2900 | Accuracy: 92%

Task 2:Model 2 (learning rate = 0.01, epochs=20, batch_size=10) Train loss: 0.0748 | Accuracy: 98% Test loss: 0.1500 | Accuracy: 96%

Task 3:Model 3 (learning rate = 0.01, epochs=20, batch_size=10) Train loss: 0.2287 | Accuracy: 90% Test loss: 0.2930 | Accuracy: 89%

Observations: It seems as though model 2 did the best is their respective iterations. However, all three models did very well as they all had accuracies around 90% with losses less than 1. It is interesting to see how each model improves throughout the epochs. For example, model 3 started at an accuracy of 70% and 86% for training and test respectively and ended up improving until they hit 90% and 89% at the 8th epoch and stayed the same throughout the rest of the runtime.
All three models took longer than I had assumed in their runtime. Model 1 ran 15 minutes, model 2 ran for 16 minutes, and model 3 ran for 27 minutes although that could just be google colab's runtime limitations. It makes sense though that the runtimes would increase over the models because the complexity increases as well.

**Part 2**

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()

        # Define various layers here, such as in the tutorial example
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.m1_conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        self.m2_conv1 = nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3)
        self.m2_conv2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3)
        #self.conv2 = Make conv layer like above
        #self.conv3 = Make conv layer like above

        self.fc1_model1 = nn.Linear(360, 100)  # This is first fully connected layer for step 1.
        self.fc1_model2 = nn.Linear(1440, 100) # This is first fully connected layer for step 2.
        self.fc1_model3 = nn.Linear(640, 100)  # This is first fully connected layer for step 3

        self.fc2 = nn.Linear(100, 10)       # This is 2nd fully connected layer for all models.

        self.fc_model0 = nn.Linear(2250, 100)   # This is for example model.


        # This will select the forward pass function based on mode for the ConvNet.
        # Based on the question, you have 3 modes available for step 1 to 3.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        elif mode == 0:
            self.forward = self.model_0
        else:
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)


    # Example model. Modify this for step 1-3
    def model_0(self, X):
        # ======================================================================

        X = F.relu(self.conv1(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        #print(X.shape)

        X = torch.flatten(X, start_dim=1)
        #print(X.shape)

        X = F.relu(self.fc_model0(X))
        X = self.fc2(X)

        return X


    # Simple CNN. step 1
    def model_1(self, X):
        # ======================================================================

        # Complete this part as model_0, add one more conv2d layer
        # with relu activation followed by maxpool layer.
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, kernel_size=2)
        X = F.relu(self.m1_conv2(X))
        X = F.max_pool2d(X,kernel_size=2)

        X = torch.flatten(X, start_dim=1)

        #X = F.relu(self.fc1_model1(X))
        X = self.fc1_model1(X)
        X = self.fc2(X)

        return X


    # Increase filters. step 2
    def model_2(self, X):
        # ======================================================================

        # Complete this part as model_1. Modify in/out channels for conv2d layers.
        X = F.relu(self.m2_conv1(X))
        X = F.max_pool2d(X, kernel_size=2)
        X = F.relu(self.m2_conv2(X))
        X = F.max_pool2d(X, kernel_size=2)

        X = torch.flatten(X, start_dim=1)

        #X = F.relu(self.fc1_model2(X))
        X = self.fc1_model2(X)
        X = self.fc2(X)

        return X


    # Large CNN. step 3
    def model_3(self, X):
        # ======================================================================

        # Complete this part as model_2, add one more conv2d layer
        # with relu activation. Do not add maxpool after this new conv2d layer.

        X = torch.flatten(X, start_dim=1)

        X = F.relu(self.fc1_model3(X))
        X = self.fc2(X)

        return X






In [ ]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
#from ConvNet import ConvNet
import argparse
import numpy as np

def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''

    # Set model to train mode before each epoch
    model.train()

    # Empty list to store losses
    losses = []
    correct = 0

    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample

        # Push data/label to correct device
        data, target = data.to(device), target.to(device)
        # print(data.shape)
        # print(target.shape)
        # exit()

        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()

        # Do forward pass for current set of data
        output = model(data)

        # ======================================================================
        # Compute loss based on criterion
        loss = criterion(output, target)


        # Computes gradient based on final loss
        loss.backward()

        # Store loss
        losses.append(loss.item())

        # Optimize model parameters based on learning rate and gradient
        optimizer.step()

        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)

        # ======================================================================
        # Count correct predictions overall
        correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc



def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''

    # Set model to eval mode to notify all layers.
    model.eval()

    losses = []
    correct = 0

    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)


            # Predict for data by doing forward pass
            output = model(data)

            # ======================================================================
            # Compute loss based on same criterion as training
            loss = F.cross_entropy(output, target, reduction='mean')

            # Append loss to overall test loss
            losses.append(loss.item())

            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)

            # ======================================================================
            # Count correct predictions overall
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))

    return test_loss, accuracy


def run_main(FLAGS):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()

    # Set proper device based on cuda availability
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Torch device selected: ", device)

    # Initialize the model and send to device
    model = ConvNet(FLAGS.mode).to(device)
    # print(model)
    # exit()

    # Initialize the criterion for loss computation
    criterion = nn.CrossEntropyLoss(reduction='mean')

    # Initialize optimizer type
    optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)

    # Create transformations to apply to each data sample
    # Can specify variations such as image flip, color flip, random crop, ...
    #transform=transforms.Compose([
    #    transforms.ToTensor(),
    #    transforms.Normalize((0.1307,), (0.3081,))
    #    ])

    transform = transforms.Compose(
                    [transforms.ToTensor(),
                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # Load datasets for training and testing
    # Inbuilt datasets available in torchvision (check documentation online)
    dataset1 = datasets.CIFAR10('./data/', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.CIFAR10('./data/', train=False,
                       transform=transform)
    train_loader = DataLoader(dataset1, batch_size = FLAGS.batch_size,
                                shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size = FLAGS.batch_size,
                                shuffle=False, num_workers=4)

    best_accuracy = 0.0

    # Run training for n_epochs specified in config
    for epoch in range(1, FLAGS.num_epochs + 1):
        print("\nEpoch: ", epoch)
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, FLAGS.batch_size)
        test_loss, test_accuracy = test(model, device, test_loader)

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy

    print("accuracy is {:2.2f}".format(best_accuracy))

    print("Training and evaluation finished")


if __name__ == '__main__':
    # Set parameters for Sparse Autoencoder
    parser = argparse.ArgumentParser('CNN Exercise.')
    parser.add_argument('--mode',
                        type=int, default=1,
                        help='Select mode between 1-3.')
    parser.add_argument('--learning_rate',
                        type=float, default=0.01,
                        help='Initial learning rate.')
    parser.add_argument('--num_epochs',
                        type=int,
                        default=10,
                        help='Number of epochs to run trainer.')
    parser.add_argument('--batch_size',
                        type=int, default=32,
                        help='Batch size. Must divide evenly into the dataset sizes.')
    parser.add_argument('--log_dir',
                        type=str,
                        default='logs',
                        help='Directory to put logging.')

    FLAGS = None
    FLAGS, unparsed = parser.parse_known_args()

    print("Mode: ", FLAGS.mode)
    print("LR: ", FLAGS.learning_rate)
    print("Batch size: ", FLAGS.batch_size)

    run_main(FLAGS)




Mode:  1
LR:  0.01
Batch size:  32
Torch device selected:  cpu
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 2.0462, Accuracy: 12727/50016 (25%)
Test set: Average loss: 1.8329, Accuracy: 3560/10000 (36%)

Epoch:  2
Train set: Average loss: 1.6998, Accuracy: 19801/50016 (40%)
Test set: Average loss: 1.5508, Accuracy: 4464/10000 (45%)

Epoch:  3
Train set: Average loss: 1.4803, Accuracy: 23658/50016 (47%)
Test set: Average loss: 1.4661, Accuracy: 4802/10000 (48%)

Epoch:  4
Train set: Average loss: 1.4000, Accuracy: 25197/50016 (50%)
Test set: Average loss: 1.3737, Accuracy: 5195/10000 (52%)

Epoch:  5
Train set: Average loss: 1.3480, Accuracy: 26202/50016 (52%)
Test set: Average loss: 1.3736, Accuracy: 5204/10000 (52%)

Epoch:  6
Train set: Average loss: 1.3037, Accuracy: 26922/50016 (54%)
Test set: Average loss: 1.3252, Accuracy: 5331/10000 (53%)

Epoch:  7
Train set: Average loss: 1.2690, Accuracy: 27585/50016 (55%)
Test set: Average loss: 1.2679, Accuracy:

Task 4:

For learning rate 0.5: Training Loss: 1.7937 | Training Accuracy: 35% Testing Loss: 1.9294 | Testing Accuracy: 30%

For learning rate 0.1: Training Loss: 1.1643 | Training Accuracy: 60% Testing Loss: 1.2428 | Testing Accuracy: 56%


For learning rate 0.01: Training Loss: 1.1494 | Training Accuracy: 60% Testing Loss: 1.1834 | Testing Accuracy: 58%

Observations: The learning rate did seem to have an impact in how the models ran. There is a pattern of the lower the learning rate, the better the model does. I could also see this happening during the runtime with each epoch. I noticed that for the learning rate of 0.5, the accuracies seemed to oscillate between being overfit and underfit and couldn't get that far in improving them while for the learning rate of 0.1, the accuracies started off differnt in the first epoch but immediately came together on the second and was consistent throughout.
Task 5:

For 10 epochs:
Training Loss: 0.7996 | Training Accuracy: 72%
Testing Loss: 1.0160 | Testing Accuracy: 65%

For 20 epochs:
Training Loss: 0.7217 | Training Accuracy: 75%
Testing Loss: 1.0194 | Testing Accuracy: 67%

For 30 epochs:
Training Loss: 0.6866 | Training Accuracy: 76%
Testing Loss: 1.0808 | Testing Accuracy: 66%

Observations: I was expecting for the higher epochs to output a better result but that does not seem to be the case in this problem in terms of accuracy. For all three iterations, the output was relatively the same except for the loss where I did notice that the losses did seem to decrease when given higher epochs to run.